In [1]:
import os

In [2]:
%pwd

'c:\\Users\\bisht\\OneDrive\\Desktop\\Personal_project\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size:list
    params_learning_rate:float
    params_include_top:bool
    params_weights:str
    params_classes:int
    params_dropout:float

In [5]:
from src.Banking_System.constants import *
from src.Banking_System.utils.common import read_yaml,create_directories
from src.Banking_System import logger

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_dropout=self.params.DROPOUT)
        return prepare_base_model_config


In [7]:
import os
import urllib.request as request
import zipfile
from src.Banking_System import logger
import tensorflow as tf
import keras

In [8]:
class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig):
        self.config=config
    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        model=self.model
        self.save_model(path=self.config.base_model_path,model=self.model)
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
    def prepare_full_model(self,model,classes,frozen_all,frozen_till,learning_rate,dropout_no):
        if frozen_all:
            for layer in model.layers:
                model.trainable=False
        elif (frozen_till is not None) and (frozen_till>0):
            for layer in model.layers[:frozen_till]:
                model.trainable=False
        flatten_in=tf.keras.layers.Flatten()(model.output)
        dense1=tf.keras.layers.Dense(units=256,activation='relu',)(flatten_in)
        Normalization=tf.keras.layers.BatchNormalization()(dense1)
        dense2=tf.keras.layers.Dense(units=128,activation='relu',kernel_initializer='he_normal')(Normalization)
        dropout=tf.keras.layers.Dropout(dropout_no)(dense2)
        dense3=tf.keras.layers.Dense(units=64,activation='relu',kernel_initializer='he_normal')(dropout)
        dropout=tf.keras.layers.Dropout(dropout_no)(dense3)
        dense4=tf.keras.layers.Dense(units=32,activation='relu',kernel_initializer='he_normal')(dropout)
        dropout=tf.keras.layers.Dropout(dropout_no)(dense4)
        dense5=tf.keras.layers.Dense(units=16,activation='relu',kernel_initializer='he_normal')(dropout)
        prediction=tf.keras.layers.Dense(units=classes,activation='softmax')(dense5)
        full_model=tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction

        )
        full_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                          loss=tf.keras.losses.CategoricalCrossentropy(),
                          metrics=['accuracy'])
        return full_model
        
    def update_base_model(self):
        self.full_model=self.prepare_full_model(
                model=self.model,
                classes=self.config.params_classes,
                frozen_all=True,
                frozen_till=None,
                learning_rate=self.config.params_learning_rate,
                dropout_no=self.config.params_dropout
            )
        full_model=self.full_model
        full_model.summary()
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)


    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
        

In [9]:
try:
    config=ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2024-03-27 21:40:07,943: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-27 21:40:07,953: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-27 21:40:07,955: INFO: common: created directory at: artifacts/prepare_base_model]


[2024-03-27 21:40:08,486: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                          

In [12]:
import sqlite3
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
credit_score='artifacts\\training\\creditscore_model.pkl'

conn=sqlite3.connect(r'C:\\Users\bisht\\OneDrive\Desktop\\Personal_project\\customer_data.db')
cur=conn.cursor()
cur.execute('Select * From Customer where username=? and password=?',('alice123', 'password1'))
conn.commit()
data=cur.fetchall()
data1=list(data[0])
indexs=[0,1,2,3,11]
data1=[value for index, value in enumerate(data1) if index not in indexs]
pred_data=pd.DataFrame([data1],columns=['Age','Income','Education','Number of Children','Gender','Marital Status','Home Ownership'])
pred_data.replace({'MALE':True,'Female':False,'Single':True,'Married':False,'Owned':False,'Rented':True},inplace=True)
oe = OrdinalEncoder(categories=[['High School Diploma', "Associate's Degree", "Bachelor's Degree", "Master's Degree", 'Doctorate']])
pred_data['Education'] = oe.fit_transform(pred_data[['Education']])
data_pred=pred_data.to_numpy()
prediction=credit_score.predict(data_pred)


C:\Users\bisht\AppData\Local\Temp\ipykernel_4564\3156896650.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pred_data.replace({'MALE':True,'Female':False,'Single':True,'Married':False,'Owned':False,'Rented':True},inplace=True)


ValueError: Found unknown categories ['Bachelor'] in column 0 during fit